In [26]:
import torch
from ofa.imagenet_classification.elastic_nn.networks import OFAProxylessNASNets
from ofa.utils import count_net_flops, count_parameters

ofa_net = OFAProxylessNASNets(
    n_classes=5,
    bn_param=(0.1, 1e-5),
    dropout_rate=0.1,
    base_stage_width="proxyless",
    width_mult=0.4,
    ks_list=[3,5],
    expand_ratio_list=[2,3,4],
    depth_list=[1,2,3],
)

model_path = "exp/kernel_depth2kernel_depth_width/phase2/checkpoint/checkpoint.pth.tar"
init = torch.load(model_path, map_location="cpu")["state_dict"]
ofa_net.load_state_dict(init)
# ofa_net = ofa_net.double()


In [27]:
from sklearn.metrics import confusion_matrix, classification_report

def eval(net, test_loader):
    net.eval()
    correct = 0
    total = 0

    y_pred = []
    y_true = []

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            output = torch.argmax(outputs, dim=1).data.cpu().numpy()
            y_pred.extend(output)  # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels)  # Save Truth

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    print(cf_matrix)
    target_names = ["class 0", "class 1", "class 2", "class 3", "class 4"]
    report = classification_report(
        y_true, y_pred, target_names=target_names, output_dict=True
    )
    print("Accuracy of the network on the test data: %.3f" % (correct / total))
    return (correct / total), report

In [28]:
from torch.utils.data import Dataset, DataLoader
from ofa.imagenet_classification.data_providers.mitbih import ECGDataset

def load_dataset(path, batch_size, shuffle, dims=2):
    dataset = ECGDataset(file_path=path, dims=dims)
    dataset_loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0
    )
    return dataset_loader

In [4]:
from ofa.utils import (
    AverageMeter,
    accuracy,
    write_log
)
from tqdm import tqdm
import torch.nn as nn

def validate(
    epoch=0,
    is_test=False,
    run_str="",
    net=None,
    data_loader=None,
    no_logs=False,
    train_mode=False,
):
    def get_metric_dict():
        return {
            "top1": AverageMeter(),
            "top5": AverageMeter(),
        }

    def update_metric(metric_dict, output, labels):
        acc1, acc5 = accuracy(output, labels, topk=(1, 5))
        metric_dict["top1"].update(acc1[0].item(), output.size(0))
        metric_dict["top5"].update(acc5[0].item(), output.size(0))

    def get_metric_vals(metric_dict, return_dict=False):
        if return_dict:
            return {key: metric_dict[key].avg for key in metric_dict}
        else:
            return [metric_dict[key].avg for key in metric_dict]

    if not isinstance(net, nn.DataParallel):
        net = nn.DataParallel(net)

    if train_mode:
        net.train()
    else:
        net.eval()

    losses = AverageMeter()
    metric_dict = get_metric_dict()

    with torch.no_grad():
        with tqdm(
            total=len(data_loader),
            desc="Validate Epoch #{} {}".format(epoch + 1, run_str),
            disable=no_logs,
        ) as t:
            for i, (images, labels) in enumerate(data_loader):
                #print(i, images, labels)
                images, labels = images.to(torch.device("cpu")), labels.to(torch.device("cpu"))
                # compute output
                output = net(images)
                loss = nn.CrossEntropyLoss()(output, labels)
                # measure accuracy and record loss
                update_metric(metric_dict, output, labels)

                losses.update(loss.item(), images.size(0))
                t.set_postfix(
                    {
                        "loss": losses.avg,
                        **get_metric_vals(metric_dict, return_dict=True),
                        "img_size": images.size(2),
                    }
                )
                t.update(1)
    return losses.avg, get_metric_vals(metric_dict)

In [29]:
from ofa.imagenet_classification.elastic_nn.utils import set_running_statistics


ofa_net.re_organize_middle_weights()
ofa_net.set_active_subnet(ks=3,e=2,d=1)

active_subnet = ofa_net.get_active_subnet()
active_subnet = active_subnet.double()
# print(ofa_net.get_active_net_config())
print("Number of params", count_parameters(active_subnet))
print("Number of FLOPS", count_net_flops(active_subnet.float(), data_shape=[1,1,1,260]))
active_subnet.double()
# Change batch_size below will only affect the result slightly, almost no difference
val_loader = load_dataset("dataset/271022/mitbih_val.csv", batch_size=512, shuffle=True)
set_running_statistics(active_subnet, val_loader)

# Change batch_size below does not affect the result at all
test_loader = load_dataset("dataset/271022/mitbih_test.csv", batch_size=512, shuffle=True)
acc, report = eval(active_subnet, test_loader)
print(acc, report)

# loss, (top1, top5) = validate(
#     epoch=0, is_test=True, data_loader=test_loader, run_str="Test Model", net=active_subnet
# )

Number of params 12437
Number of FLOPS 180272.0
[[13209   299    92    75    39]
 [   22   392     8     1     0]
 [   12    10  1077    12     4]
 [    6     0     4   114     1]
 [    9     0     7     1  1147]]
Accuracy of the network on the test data: 0.964
0.9636055861193398 {'class 0': {'precision': 0.9963041182682154, 'recall': 0.9631763161732536, 'f1-score': 0.9794601809283701, 'support': 13714}, 'class 1': {'precision': 0.5592011412268189, 'recall': 0.9267139479905437, 'f1-score': 0.697508896797153, 'support': 423}, 'class 2': {'precision': 0.9065656565656566, 'recall': 0.9659192825112107, 'f1-score': 0.9353017802865826, 'support': 1115}, 'class 3': {'precision': 0.5615763546798029, 'recall': 0.912, 'f1-score': 0.6951219512195123, 'support': 125}, 'class 4': {'precision': 0.963056255247691, 'recall': 0.9853951890034365, 'f1-score': 0.9740976645435245, 'support': 1164}, 'accuracy': 0.9636055861193398, 'macro avg': {'precision': 0.797340705197637, 'recall': 0.9506409471356889, '

In [23]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report

from ofa.imagenet_classification.elastic_nn.networks import OFAProxylessNASNets
from ofa.imagenet_classification.data_providers.mitbih import ECGDataset
from ofa.utils import count_net_flops, count_parameters
from ofa.imagenet_classification.elastic_nn.utils import set_running_statistics


def eval(net, test_loader):
    net.eval()
    correct = 0
    total = 0

    y_pred = []
    y_true = []

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            output = torch.argmax(outputs, dim=1).data.cpu().numpy()
            y_pred.extend(output)  # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels)  # Save Truth

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    print(cf_matrix)
    target_names = ["class 0", "class 1", "class 2", "class 3", "class 4"]
    report = classification_report(
        y_true, y_pred, target_names=target_names, output_dict=True
    )
    print("Accuracy of the network on the test data: %.3f" % (correct / total))
    return (correct / total), report

def load_dataset(path, batch_size, shuffle, dims=2):
    dataset = ECGDataset(file_path=path, dims=dims)
    dataset_loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0
    )
    return dataset_loader

def get_bin_id(total_macs):
    MIN_MACS = 180000
    MAX_MACS = 940000
    if total_macs < MIN_MACS:
        return 0
    elif total_macs > MAX_MACS:
        return 11
    else:
        return (int) ((10) * (total_macs - MIN_MACS) / (MAX_MACS - MIN_MACS))

ofa = OFAProxylessNASNets(
    n_classes=5,
    bn_param=(0.1, 1e-5),
    dropout_rate=0.1,
    base_stage_width="proxyless",
    width_mult=0.4,
    ks_list=[3,5],
    expand_ratio_list=[2,3,4],
    depth_list=[1,2,3],
)

model_path = "exp/kernel_depth2kernel_depth_width/phase2/checkpoint/checkpoint.pth.tar"
init = torch.load(model_path, map_location="cpu")["state_dict"]
ofa.load_state_dict(init)
ofa.re_organize_middle_weights()

# Change batch_size below will only affect the result slightly, almost no difference
val_loader = load_dataset("dataset/271022/mitbih_val.csv", batch_size=512, shuffle=True)

# Change batch_size below does not affect the result at all
test_loader = load_dataset("dataset/271022/mitbih_test.csv", batch_size=512, shuffle=True)


macs_list = []
param_list = []
acc_list = []
report_acc_list = []
report_macro_precision_list = []
report_macro_recall_list = []
report_macro_f1_list = []
report_wa_precision_list = []
report_wa_recall_list = []
report_wa_f1_list = []

model_config_list = []

model = ofa.get_active_subnet()
model.double()
set_running_statistics(model, val_loader)

TOTAL_MACS_SUPERNETWORK = count_net_flops(model.float(), data_shape=[1, 1, 1, 260])
print("TOTAL_MACS_SUPERNETWORK", TOTAL_MACS_SUPERNETWORK)
model.double()
acc, report = eval(model, test_loader)

count = 0
NUM_OF_SAMPLES = 100
macs_histogram = {}

while count < NUM_OF_SAMPLES:
    if count != 0:
        model_config = ofa.sample_active_subnet()
        print(model_config)
        ofa.set_active_subnet(
            model_config["ks"],
            model_config["e"],
            model_config["d"],
        )
    else:
        model_config = None
    
    
    model = ofa.get_active_subnet()
    model.double()
    set_running_statistics(model, val_loader)
    total_macs = count_net_flops(model.float(), data_shape=[1, 1, 1, 260])

    bin_id = get_bin_id(total_macs) 
    if bin_id in macs_histogram and macs_histogram[bin_id]>= NUM_OF_SAMPLES / 10:
        print("Continue")
        continue
        
    elif bin_id not in macs_histogram:
        macs_histogram[bin_id] = 1
    else:
        macs_histogram[bin_id] += 1

    model.double()

    print(model_config)
    total_params = count_parameters(model)
    print("total_macs", total_macs)
    print("total_params", total_params)
    acc, report = eval(model, test_loader)

    report_macro_precision_list.append(report['macro avg']['precision'])
    report_macro_recall_list.append(report['macro avg']['recall'])
    report_macro_f1_list.append(report['macro avg']['f1-score'])
    report_wa_precision_list.append(report['weighted avg']['precision'])
    report_wa_recall_list.append(report['weighted avg']['recall'])
    report_wa_f1_list.append(report['weighted avg']['f1-score'])
    report_acc_list.append(report['accuracy'])

    macs_list.append(total_macs)
    param_list.append(total_params)
    acc_list.append(acc)
    model_config_list.append(model_config)
    count += 1

    df_data_dict = {
        "total_macs": macs_list,
        "total_params": param_list,
        "acc": acc_list,
        "report_acc": report_acc_list,
        "macro_precision": report_macro_precision_list,
        "macro_recall": report_macro_recall_list,
        "macro_f1": report_macro_f1_list,
        "wa_precision": report_wa_precision_list,
        "wa_recall": report_wa_recall_list,
        "wa_f1": report_wa_f1_list,
        "configs": model_config_list,
    }
    pd.DataFrame.from_dict(df_data_dict).to_csv("acc_versus_size_ofa_proxyless.csv")

TOTAL_MACS_SUPERNETWORK 938560.0
[[13555    95    30    21    13]
 [   17   402     4     0     0]
 [   10     3  1090     9     3]
 [    9     0     4   112     0]
 [    7     0     2     0  1155]]
Accuracy of the network on the test data: 0.986
None
total_macs 938560.0
total_params 55525
[[13555    95    30    21    13]
 [   17   402     4     0     0]
 [   10     3  1090     9     3]
 [    9     0     4   112     0]
 [    7     0     2     0  1155]]
Accuracy of the network on the test data: 0.986
{'ks': [3, 3, 5, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 3, 5, 5], 'e': [4, 4, 3, 3, 3, 4, 3, 2, 2, 2, 3, 4, 4, 3, 3, 2], 'd': [1, 3, 1, 2, 1, 1]}
{'ks': [3, 3, 5, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 3, 5, 5], 'e': [4, 4, 3, 3, 3, 4, 3, 2, 2, 2, 3, 4, 4, 3, 3, 2], 'd': [1, 3, 1, 2, 1, 1]}
total_macs 359064.0
total_params 21725
[[13402   197    46    48    21]
 [   12   402     8     1     0]
 [   14     4  1083    12     2]
 [    5     0     6   114     0]
 [    4     1     5     0  1154]]
Accuracy of the 

KeyboardInterrupt: 

In [4]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report

from ofa.imagenet_classification.elastic_nn.networks import OFAProxylessNASNets
from ofa.imagenet_classification.data_providers.mitbih import ECGDataset
from ofa.utils import count_net_flops, count_parameters
from ofa.imagenet_classification.elastic_nn.utils import set_running_statistics


def eval(net, test_loader):
    net.eval()
    correct = 0
    total = 0

    y_pred = []
    y_true = []

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            output = torch.argmax(outputs, dim=1).data.cpu().numpy()
            y_pred.extend(output)  # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels)  # Save Truth

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    print(cf_matrix)
    target_names = ["class 0", "class 1", "class 2", "class 3", "class 4"]
    report = classification_report(
        y_true, y_pred, target_names=target_names, output_dict=True
    )
    print("Accuracy of the network on the test data: %.3f" % (correct / total))
    return (correct / total), report

def load_dataset(path, batch_size, shuffle, dims=2):
    dataset = ECGDataset(file_path=path, dims=dims)
    dataset_loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0
    )
    return dataset_loader

def get_bin_id(total_macs):
    MIN_MACS = 180000
    MAX_MACS = 940000
    if total_macs < MIN_MACS:
        return 0
    elif total_macs > MAX_MACS:
        return 11
    else:
        return (int) ((10) * (total_macs - MIN_MACS) / (MAX_MACS - MIN_MACS))

ofa = OFAProxylessNASNets(
    n_classes=5,
    bn_param=(0.1, 1e-5),
    dropout_rate=0.1,
    base_stage_width="proxyless",
    width_mult=0.4,
    ks_list=[3,5],
    expand_ratio_list=[2,3,4],
    depth_list=[1,2,3],
)

model_path = "exp/kernel_depth2kernel_depth_width/phase2/checkpoint/checkpoint.pth.tar"
init = torch.load(model_path, map_location="cpu")["state_dict"]
ofa.load_state_dict(init)
ofa.re_organize_middle_weights()

# Change batch_size below will only affect the result slightly, almost no difference
val_loader = load_dataset("dataset/271022/mitbih_val.csv", batch_size=512, shuffle=True)

# Change batch_size below does not affect the result at all
test_loader = load_dataset("dataset/271022/mitbih_test.csv", batch_size=512, shuffle=True)


macs_list = []
param_list = []
acc_list = []
report_acc_list = []
report_macro_precision_list = []
report_macro_recall_list = []
report_macro_f1_list = []
report_wa_precision_list = []
report_wa_recall_list = []
report_wa_f1_list = []

model_config_list = []


count = 1
NUM_OF_SAMPLES = 2
macs_histogram = {}

while count < NUM_OF_SAMPLES:
    model_config = {'ks': [5, 5, 5, 5, 5, 3, 3, 3, 3, 5, 3, 3, 5, 5, 3, 5], 'e': [3, 4, 3, 4, 4, 3, 2, 2, 2, 3, 4, 2, 4, 2, 3, 3], 'd': [3, 3, 3, 2, 3, 1]}
    if count != 0:
        # model_config = ofa.sample_active_subnet()
        print(model_config)
        ofa.set_active_subnet(
            model_config["ks"],
            model_config["e"],
            model_config["d"],
        )
    else:
        model_config = None
    
    
    model = ofa.get_active_subnet()
    model.double()
    set_running_statistics(model, val_loader)
    total_macs = count_net_flops(model.float(), data_shape=[1, 1, 1, 260])

    model.double()

    print(model_config)
    total_params = count_parameters(model)
    print("total_macs", total_macs)
    print("total_params", total_params)
    acc, report = eval(model, test_loader)

    report_macro_precision_list.append(report['macro avg']['precision'])
    report_macro_recall_list.append(report['macro avg']['recall'])
    report_macro_f1_list.append(report['macro avg']['f1-score'])
    report_wa_precision_list.append(report['weighted avg']['precision'])
    report_wa_recall_list.append(report['weighted avg']['recall'])
    report_wa_f1_list.append(report['weighted avg']['f1-score'])
    report_acc_list.append(report['accuracy'])

    macs_list.append(total_macs)
    param_list.append(total_params)
    acc_list.append(acc)
    model_config_list.append(model_config)
    count += 1

    df_data_dict = {
        "total_macs": macs_list,
        "total_params": param_list,
        "acc": acc_list,
        "report_acc": report_acc_list,
        "macro_precision": report_macro_precision_list,
        "macro_recall": report_macro_recall_list,
        "macro_f1": report_macro_f1_list,
        "wa_precision": report_wa_precision_list,
        "wa_recall": report_wa_recall_list,
        "wa_f1": report_wa_f1_list,
        "configs": model_config_list,
    }
    print(df_data_dict)

{'ks': [5, 5, 5, 5, 5, 3, 3, 3, 3, 5, 3, 3, 5, 5, 3, 5], 'e': [3, 4, 3, 4, 4, 3, 2, 2, 2, 3, 4, 2, 4, 2, 3, 3], 'd': [3, 3, 3, 2, 3, 1]}
{'ks': [5, 5, 5, 5, 5, 3, 3, 3, 3, 5, 3, 3, 5, 5, 3, 5], 'e': [3, 4, 3, 4, 4, 3, 2, 2, 2, 3, 4, 2, 4, 2, 3, 3], 'd': [3, 3, 3, 2, 3, 1]}
total_macs 658448.0
total_params 37013
[[13497   137    30    34    16]
 [   18   400     5     0     0]
 [   13     1  1087    11     3]
 [    8     0     4   113     0]
 [    6     0     2     0  1156]]
Accuracy of the network on the test data: 0.983
{'total_macs': [658448.0], 'total_params': [37013], 'acc': [0.9825887189408138], 'report_acc': [0.9825887189408138], 'macro_precision': [0.880568714317363], 'macro_recall': [0.9603636542733488], 'macro_f1': [0.9158268849382871], 'wa_precision': [0.9849450281331457], 'wa_recall': [0.9825887189408138], 'wa_f1': [0.9833379650484863], 'configs': [{'ks': [5, 5, 5, 5, 5, 3, 3, 3, 3, 5, 3, 3, 5, 5, 3, 5], 'e': [3, 4, 3, 4, 4, 3, 2, 2, 2, 3, 4, 2, 4, 2, 3, 3], 'd': [3, 3, 3, 2